In [4]:
import sagas.nlu.tts_utils as tts
tts.say_lang("私はサッカーをします", 'ja')

speak language with ja ...
done.


In [34]:
from sagas.ja.dependency import JaDeps
deps=JaDeps()
deps.tree("私はサッカーをします")

      私は---D
  サッカーを-D
        します
EOS
* 0 2D 0/1 -2.389876
私	名詞,代名詞,一般,*,*,*,私,ワタシ,ワタシ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
* 1 2D 0/1 -2.389876
サッカー	名詞,一般,*,*,*,*,サッカー,サッカー,サッカー
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
* 2 -1D 0/1 0.000000
し	動詞,自立,*,*,サ変・スル,連用形,する,シ,シ
ます	助動詞,*,*,*,特殊・マス,基本形,ます,マス,マス
EOS

% 私はサッカーをします
✆ shi(actor:+私@n, obj:+サッカー@n, act:*する@v)
ﺴ watashi ha sakka- wo shi masu


### 如何获得依赖信息
依赖关系具有树结构。

- bnst.parent返回父子句。但是，返回None是因为最后一个子句没有父子句。
- bnst.children返回子子句列表。有时没有孩子（空列表）或多个孩子。

下面的代码用arrow->打印每个依赖关系

In [23]:
from pyknp import KNP
# from pyknp import Juman

knp = KNP()
result = knp.parse("望遠鏡で泳いでいる少女を見た。")

def dump_deps(result):
    for bnst in result.bnst_list():
        parent = bnst.parent
        if parent is not None:
            child_rep = " ".join(mrph.repname for mrph in bnst.mrph_list())
            parent_rep = " ".join(mrph.repname for mrph in parent.mrph_list())
            print(child_rep, "->", parent_rep)

dump_deps(result)

望遠/ぼうえん 鏡/きょう  -> 見る/みる 
泳ぐ/およぐ いる/いる -> 少女/しょうじょ 
少女/しょうじょ  -> 見る/みる 


In [22]:
result = knp.parse("私は望遠鏡で泳いでいる少女を見た。")
dump_deps(result)

私/わたし  -> 見る/みる 
望遠/ぼうえん 鏡/きょう  -> 見る/みる 
泳ぐ/およぐ いる/いる -> 少女/しょうじょ 
少女/しょうじょ  -> 見る/みる 


In [21]:
result = knp.parse("私は海で泳いでいる少女を見た。")
dump_deps(result)

私/わたし  -> 見る/みる 
海/かい  -> 泳ぐ/およぐ いる/いる
泳ぐ/およぐ いる/いる -> 少女/しょうじょ 
少女/しょうじょ  -> 見る/みる 


In [44]:
from pyknp import Juman
jumanpp = Juman()
test_str = "この文を解析してください。"
result = jumanpp.analysis(test_str)
print(''.join(mrph.midasi for mrph in result))
print(",".join(mrph.genkei for mrph in result.mrph_list()))
# dump_deps(result)

この文を解析してください。
この,文,を,解析,する,くださる,。


### 如何获得谓词参数结构信息
谓词术语结构是由什么做什么及其术语的谓词组成的结构。KNP中的谓词期限结构信息作为基本短语级别信息保存。在pyknp中，Tag类的pas对象保存此信息，并且它指向谓词的基本子句中的Pas对象和其他基本划分中的None。

Pas对象管理有关谓词和相应谓词的术语的信息。Pas对象中的arguments对象是一个字典{case：Argument objects列表}。Argument对象管理术语的信息。案例和期限信息不是一对一的对应关系，因为案例等可以采用多个术语。

有关Pas和Argument类的详细信息，请参阅以下文档的Pas模块项。 [标签模块](https://pyknp.readthedocs.io/en/latest/tag.html)

以下是一个示例程序，用于检索句子中的谓词和术语。

In [26]:
from pyknp import KNP

knp = KNP()

def proc_predicates(sent):
    result = knp.parse(sent)
    predicates(result)
    
def predicates(result):
    for tag in result.tag_list():
        if tag.pas is not None: # find predicate
            print('述語: %s' % ''.join(mrph.midasi for mrph in tag.mrph_list()))
            for case, args in tag.pas.arguments.items(): # case: str, args: list of Argument class
                for arg in args: # arg: Argument class
                    print('\t格: %s,  項: %s  (項の基本句ID: %d)' % (case, arg.midasi, arg.tid))

proc_predicates("望遠鏡で泳いでいる少女を見た。")

述語: 泳いでいる
	格: ガ,  項: 少女  (項の基本句ID: 3)
述語: 見た。
	格: ヲ,  項: 少女  (項の基本句ID: 3)
	格: デ,  項: 鏡  (項の基本句ID: 1)


In [27]:
proc_predicates('これは日本語で何と言いますか？')

述語: 言いますか？
	格: ガ,  項: これ  (項の基本句ID: 0)
	格: デ,  項: 語  (項の基本句ID: 2)
	格: 修飾,  項: 何と  (項の基本句ID: 3)


⊕ [日语的格现象（其一） - 知乎](https://zhuanlan.zhihu.com/p/25490919)
但“补语”是一个相当模糊的统称，所以在区分不同的 NP 时我们还需要一些更加精确的术语。

- 日语里的 NP 通常都有助词跟随，我们称之为「格助詞」（case particle）。我们可以简单地用该格助词来称呼这个 NP。比如在「私が日本酒を飲む」这个句子里，「私（が）」可以称为「ガ格 NP」，「日本酒（を）」称为「ヲ格 NP」。
- 另外，我们还有一组主要在跨语言研究中使用的术语。在这组来源于拉丁语的术语中，日语「ガ格」对应的格被称为「主格」（nominative case），「ヲ格」对应的格被称为「対格」（accusative case）。此外还有「ニ格」对应的「与格」（dative case）、「ノ格」对应的「属格」（genitive case）等。主格之外的格还可以统称为「斜格」（oblique case）。
- 上面两类术语主要关注 NP 的表面形态，而下面要介绍的术语更靠近 NP 的实际意义一些——「項」（argument）。「項」的定义虽然和“补语”近似，但它的所指范围更狭窄一些，只指那些“义务性”的、必须的成分。比如在 I drink sake at home every night 这个例子里，像 I 和 sake 这样的参与者，就称为谓语动词 drink 的「項」，他们是这个事件最重要的参与者；而 home 虽然也是 NP，但它是由 at 伴随出现的，是副词性成分，并不是在「項」。而在日语中，几乎每个 NP 都有格助词伴随，所以在日语研究中一般笼统地把它们都称为「項」（和原本的定义多少有出入）。比如「私が居酒屋で日本酒を飲む」这个例子里，「私」「居酒屋」「日本酒」都是「飲む」的「項」。但是！「居酒屋」并不是这个事件的必须参与者（“喝”这个行为里，去掉“喝的地方”并不影响表述，但去掉了“喝的人”和“喝的东西”就不成立了），所以它并不是「必須項」。而「私」「日本酒」这样的「必須項」，我们把它们分别称作「外項」和「内項」。

In [5]:
import codecs
import optparse
import pyknp
import six
import sys

def draw_tree(bl, outf):
    assert isinstance(bl, pyknp.BList)

    tl = pyknp.TList()
    tags = bl.tag_list()
    for tag in tags:
        tl.push_tag(tag)
    nodelines = tl.sprint_tree().split(u"\n")[:-1]

    outf.write(u"# S-ID: %s\n" % bl.sid)

    for i, nl in enumerate(nodelines):
        outf.write(nl)
        outf.write(u"\t")
        outf.write(u"/ ")
        pas = tags[i].features.pas
        if pas is not None:
            outf.write(pas.cfid)
            outf.write(u"\t")
            for casename, arglist in sorted(pas.arguments.items()):
                for arg in arglist:
                    if arg.sid == bl.sid:
                        if hasattr(tags[arg.tid], 'head_prime_repname') \
                                and tags[arg.tid].head_prime_repname:
                            rep = tags[arg.tid].head_prime_repname
                        else:
                            rep = tags[arg.tid].repname
                    else:
                        rep = u"<sid=%s,tid=%s>" % (arg.sid, arg.tid)
                    outf.write(u"%s:%s " % (casename, rep))

        outf.write(u"/ ")
        rels = tags[i].features.rels
        if rels is not None:
            for rel in sorted(rels, key=lambda x: x.atype):
                outf.write(u"%s:%s " % (rel.atype, rel.target))
        outf.write(u"\n")

    outf.write(u"\n\n")


def draw_trees(inf, outf, lattice_format):
    juman_format = pyknp.JUMAN_FORMAT.DEFAULT
    if lattice_format:
        juman_format = pyknp.JUMAN_FORMAT.LATTICE_TOP_ONE
    lines = []
    for line in inf:
        lines.append(line)
        if line == u"EOS\n":
            bl = pyknp.BList(u"".join(lines), juman_format=juman_format)
            draw_tree(bl, outf)
            lines = []

In [9]:
outf = sys.stdout
inf=["インストールが終わったら動作チェックをします。"]
outf.write('start ..')
# draw_trees(inf, outf, False)

start ..

In [36]:
knp = KNP()
result = knp.parse("望遠鏡で泳いでいる少女を見た。")
draw_tree(result, outf)

# S-ID: 1
   望遠n┐　　　	/ / 
    鏡nでp──┐	/ / 
泳いでvいるs┐│	/ 泳ぐ/およぐ:動16	ガ:少女/しょうじょ / 
      少女nをp┤	/ / 
        見たv。*	/ 見る/みる:動1	デ:鏡/きょう?鏡/かがみ ヲ:少女/しょうじょ / 




In [39]:
def procs(sent):
    result = knp.parse(sent)
    predicates(result)
    dump_deps(result)
    draw_tree(result, outf)

procs('これは日本語で何と言いますか？')

述語: 言いますか？
	格: ガ,  項: これ  (項の基本句ID: 0)
	格: デ,  項: 語  (項の基本句ID: 2)
	格: 修飾,  項: 何と  (項の基本句ID: 3)
これ/これ  -> 言う/いう ます/ます  
日本/にほん 語/ご  -> 言う/いう ます/ます  
何と/なんと -> 言う/いう ます/ます  
# S-ID: 4
これdはp───┐	/ / 
     日本C┐　│	/ / 
      語nでp─┤	/ / 
         何とa┤	/ / 
言いvますsかp？*	/ 言う/いう:動4	ガ:これ/これ デ:語/ご 修飾:何と/なんと / 




In [41]:
procs('このお店まではどう行きますか？')

述語: 行きますか？
	格: マデ,  項: 店  (項の基本句ID: 1)
	格: 修飾,  項: どう  (項の基本句ID: 2)
この/この -> 御/お 店/みせ  
御/お 店/みせ   -> 行く/いく ます/ます  
どう/どう -> 行く/いく ます/ます  
# S-ID: 6
       このd┐　　	/ / 
おp店nまでpはp─┐	/ / 
           どうd┤	/ / 
  行きvますsかp？*	/ 行く/いく:動9	マデ:店/みせ?店/てん 修飾:どう/どう / 




In [32]:
procs('その部屋でインターネットは使えますか？')

述語: 使えますか？
	格: ガ,  項: インターネット  (項の基本句ID: 2)
	格: デ,  項: 部屋  (項の基本句ID: 1)
その/その -> 部屋/部屋 
部屋/部屋  -> 使える/つかえる ます/ます  
インターネット/いんたーねっと  -> 使える/つかえる ます/ます  
# S-ID: 7
         そのd┐　　	/ / 
        部屋Cでp─┐	/ / 
インターネットnはp┤	/ / 
    使えvますsかp？*	/ 使える/つかえる:動1	ガ:インターネット/いんたーねっと デ:部屋/部屋 / 




In [19]:
# ⊕ [jumanpp/python_knp.py at master · ku-nlp/jumanpp](https://github.com/ku-nlp/jumanpp/blob/master/sample/python_knp.py)

from pyknp import KNP
import sys
import codecs

# Use Juman with KNP
knp_with_juman = KNP()
result = knp_with_juman.parse(u"なのはがなのはなとごっちゃなのはよみにくい")
print(','.join(mrph.midasi for bunsetsu in result for mrph in bunsetsu.mrph_list()))

なのは,が,なのはな,と,ごっちゃ,な,の,は,よみ,にくい


In [13]:
# Use Juman++ with KNP
knp_with_jumanpp = KNP(jumancommand="jumanpp")
result = knp_with_jumanpp.parse(u"なのはがなのはなとごっちゃなのはよみにくい")
print(','.join(mrph.midasi for bunsetsu in result for mrph in bunsetsu.mrph_list() ))

なのは,が,なのはな,と,ごっちゃ,な,の,は,よみ,にくい


In [18]:
result = knp_with_jumanpp.parse(u"なのはがなのはなとごっちゃなのはよみにくい")
print(','.join(mrph.midasi for bunsetsu in result for mrph in bunsetsu.mrph_list() ))

なのは,が,なのはな,と,ごっちゃ,な,の,は,よみ,にくい


In [14]:
# loop for tag (kihonku, basic phrase)
for tag in result.tag_list():
    print(u"ID:%s, 見出し:%s, 係り受けタイプ:%s, 親文節ID:%s, 素性:%s" \
    % (tag.tag_id, "".join(mrph.midasi for mrph in tag.mrph_list()),
    tag.dpndtype, tag.parent_id, tag.fstring))

ID:0, 見出し:なのはが, 係り受けタイプ:D, 親文節ID:3, 素性:<文頭><人名><ガ><助詞><体言><係:ガ格><区切:0-0><格要素><連用要素><名詞項候補><先行詞候補><SM-人><SM-主体><正規化代表表記:なのは/なのは><解析格:ガ>
ID:1, 見出し:なのはなと, 係り受けタイプ:P, 親文節ID:2, 素性:<ト><助詞><体言><係:ト格><並キ:名><区切:1-2><並列タイプ:AND><格要素><連用要素><名詞項候補><先行詞候補><正規化代表表記:菜の花/なのはな>
ID:2, 見出し:ごっちゃなのは, 係り受けタイプ:D, 親文節ID:3, 素性:<ハ><助詞><体言><用言:判><係:未格><レベル:B><区切:5-5><ID:〜の〜><提題受:10><格要素><連用要素><状態述語><判定詞><名詞項候補><先行詞候補><正規化代表表記:ごっちゃ/ごっちゃ><用言代表表記:ごっちゃ/ごっちゃ><時制-無時制><格解析結果:ごっちゃ/ごっちゃ:判0:ガ/U/-/-/-/-><解析格:ヲ>
ID:3, 見出し:よみにくい, 係り受けタイプ:D, 親文節ID:-1, 素性:<文末><準否定表現><態:可能><〜にくい><用言:動><レベル:C><区切:5-5><ID:（文末）><提題受:30><主節><動態述語><正規化代表表記:読む/よむ?詠む/よむ><用言代表表記:読む/よむ+にくい/にくい?詠む/よむ+にくい/にくい><時制-未来><主題格:一人称優位><格関係0:ガ:なのは><格関係2:ヲ:ごっちゃ><格解析結果:詠む/よむ:動2:ガ/C/なのは/0/0/1;ヲ/N/なのはな/1/0/1;ヲ/N/ごっちゃ/2/0/1;ニ/U/-/-/-/-;ト/U/-/-/-/-;デ/U/-/-/-/-;カラ/U/-/-/-/-;ヨリ/U/-/-/-/-;時間/U/-/-/-/-;外の関係/U/-/-/-/-;ノ/U/-/-/-/-;修飾/U/-/-/-/-;トスル/U/-/-/-/->


In [15]:
# loop for mrph
for mrph in result.mrph_list():
    print(u"見出し:%s, 読み:%s, 原形:%s, 品詞:%s, 品詞細分類:%s, 活用型:%s, 活用形:%s, 意味情報:%s, 代表表記:%s" \
    % (mrph.midasi, mrph.yomi, mrph.genkei, mrph.hinsi, mrph.bunrui,
    mrph.katuyou1, mrph.katuyou2, mrph.imis, mrph.repname))

見出し:なのは, 読み:なのは, 原形:なのは, 品詞:名詞, 品詞細分類:人名, 活用型:*, 活用形:*, 意味情報:自動獲得:Wikipedia Wikipedia人名 Wikipedia名 Wikipedia多義 疑似代表表記 代表表記:なのは/なのは, 代表表記:なのは/なのは
見出し:が, 読み:が, 原形:が, 品詞:助詞, 品詞細分類:格助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:なのはな, 読み:なのはな, 原形:なのはな, 品詞:名詞, 品詞細分類:普通名詞, 活用型:*, 活用形:*, 意味情報:代表表記:菜の花/なのはな カテゴリ:植物 ドメイン:料理・食事, 代表表記:菜の花/なのはな
見出し:と, 読み:と, 原形:と, 品詞:助詞, 品詞細分類:格助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:ごっちゃ, 読み:ごっちゃ, 原形:ごっちゃ, 品詞:名詞, 品詞細分類:普通名詞, 活用型:*, 活用形:*, 意味情報:代表表記:ごっちゃ/ごっちゃ 副詞識別 自動獲得:テキスト, 代表表記:ごっちゃ/ごっちゃ
見出し:な, 読み:な, 原形:だ, 品詞:判定詞, 品詞細分類:*, 活用型:判定詞, 活用形:ダ列基本連体形, 意味情報:NIL, 代表表記:
見出し:の, 読み:の, 原形:のだ, 品詞:助動詞, 品詞細分類:*, 活用型:ナ形容詞, 活用形:語幹, 意味情報:NIL, 代表表記:
見出し:は, 読み:は, 原形:は, 品詞:助詞, 品詞細分類:副助詞, 活用型:*, 活用形:*, 意味情報:NIL, 代表表記:
見出し:よみ, 読み:よみ, 原形:よむ, 品詞:動詞, 品詞細分類:*, 活用型:子音動詞マ行, 活用形:基本連用形, 意味情報:代表表記:詠む/よむ ドメイン:文化・芸術, 代表表記:詠む/よむ
見出し:にくい, 読み:にくい, 原形:にくい, 品詞:接尾辞, 品詞細分類:形容詞性述語接尾辞, 活用型:イ形容詞アウオ段, 活用形:基本形, 意味情報:代表表記:にくい/にくい 反義:接尾辞-形容詞性述語接尾辞:やすい/やすい, 代表表記:にくい/にくい
